In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import transforms
from torchinfo import summary
import numpy as np
import gym
import os

from models import MDRNNCell, VAE, Controller

In [2]:
dir_path = './results/'
files = os.listdir(dir_path)
files = [f for f in files if 'best_1_1_G' in f]

num = -1
idx = 0
file = ''
for idx, fi in enumerate(files):
    Gnum = fi.split('G')[1].split('.')[0]
    Gnum = int(Gnum)
    if Gnum > num:
        file = fi
        num = Gnum
        
file_path = dir_path + file
s = torch.load(file_path)

In [3]:
ASIZE, LSIZE, RSIZE, RED_SIZE, SIZE = 3, 32, 256, 64, 64

vae = VAE(3, LSIZE, 1024) # VAE：3チャンネル、潜在ベクトルサイズ、中間層サイズ
mdrnn = MDRNNCell(LSIZE, ASIZE, RSIZE, 5) # MDRNN：潜在ベクトルサイズ、アクションサイズ、中間層サイズ、混合ガウス分布の分布数
controller = Controller(LSIZE, RSIZE, ASIZE) # コントローラー：潜在ベクトルサイズ、中間層サイズ、アクションサイズ

vae.load_state_dict(s['vae'])
mdrnn.load_state_dict(s['mdrnn'])
controller.load_state_dict(s['controller'])

<All keys matched successfully>

In [4]:
summary(vae, (1, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
VAE                                      [1, 3, 64, 64]            --
├─Encoder: 1-1                           [1, 32]                   --
│    └─Conv2d: 2-1                       [1, 32, 31, 31]           1,568
│    └─Conv2d: 2-2                       [1, 64, 14, 14]           32,832
│    └─Conv2d: 2-3                       [1, 128, 6, 6]            131,200
│    └─Conv2d: 2-4                       [1, 256, 2, 2]            524,544
│    └─Linear: 2-5                       [1, 32]                   32,800
│    └─Linear: 2-6                       [1, 32]                   32,800
├─Decoder: 1-2                           [1, 3, 64, 64]            --
│    └─Linear: 2-7                       [1, 1024]                 33,792
│    └─ConvTranspose2d: 2-8              [1, 128, 5, 5]            3,276,928
│    └─ConvTranspose2d: 2-9              [1, 64, 13, 13]           204,864
│    └─ConvTranspose2d: 2-10             [1,

In [54]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((RED_SIZE, RED_SIZE)),
    transforms.ToTensor()
])

In [55]:
i = 0
early_termination = True
time_limit = 2000
imgs = []

def get_action_and_transition(obs, hidden):
        """ 行動を起こし、遷移

        VAEを用いて観測値を潜在状態に変換し、MDRNNを用いて次の潜在状態と次の隠れ状態の推定を行い、コントローラに対応するアクションを計算する。

        :args obs: current observation (1 x 3 x 64 x 64) torch tensor
        :args hidden: current hidden state (1 x 256) torch tensor

        :returns: (action, next_hidden)
            - action: 1D np array
            - next_hidden (1 x 256) torch tensor
        """
        _, latent_mu, _ = vae(obs) # latent_mu: 潜在ベクトルの期待値

        action = controller(latent_mu, hidden[0] ) # コントローラーによるアクションの計算

        mus, sigmas, logpi, rs, d, next_hidden = mdrnn(action, latent_mu, hidden) # MDRNNによる次の潜在状態と次の隠れ状態の推定

        return action.squeeze().cpu().numpy(), next_hidden


with torch.no_grad():          
    env = gym.make('CarRacing-v2', render_mode='rgb_array', domain_randomize=False) # 環境：CarRacing-v2

    obs, _ = env.reset() # 環境のリセット
    imgs.append(obs) # 画像の取得

    hidden = [
        torch.zeros(1, RSIZE)#.to(device) # 隠れ状態の初期化
        for _ in range(2)]

    neg_count = 0 # 負の報酬を受け取った回数

    cumulative = 0 # 累積報酬
    i = 0
    while True:
        obs = transform(obs).unsqueeze(0)#.to(device) # 観測（画像）の前処理：obs(1, 3, 64, 64)
        
        action, hidden = get_action_and_transition(obs, hidden) # 行動を起こし、遷移：action(1, ASIZE), hidden(1, RSIZE)
        #Steering: Real valued in [-1, 1] 
        #Gas: Real valued in [0, 1]
        #Break: Real valued in [0, 1]

        obs, reward, done, _, _ = env.step(action) # 行動を実行し、報酬を受け取る：obs(3, 64, 64), reward, done, info
        imgs.append(obs) # 画像の取得
        
        #報酬を得られなかった（コース外に出たなど）連続回数をカウント
        neg_count = neg_count+1 if reward < 0.0 else 0 
        
        #トレーニングのスピードアップのために、コース外の評価を行い，20time step以上コース外に出た場合はロールアウトを終了する
        if (neg_count>20 and early_termination):  
            done = True
        
        cumulative += reward # 累積報酬の更新
        
        # ロールアウトの終了：タイムリミットに達した場合、早期終了した場合, 完了した場合
        if done or (early_termination and i > time_limit):
            env.close()
            break

        i += 1

c:\Users\daiki\miniconda3\envs\env_torch_world_models\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [56]:
cumulative

902.799999999984

In [57]:
import imageio

imageio.mimsave('./results/rollout.gif', imgs, duration=20)